# Homework 2
In homework 2, we will work with log content of user in 30 days. The general idea here is we first try to analyze one day
then apply it to a month. Thanks to **pyspark**  supporting glob pattern directory, this can be done with ease. 


In [1]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col, to_date, when, \
	input_file_name, regexp_extract, sum, lit

In [2]:
builder = (
	SparkSession.builder.appName("homework1")
	.config("spark.driver.memory", "16g")
	.config("spark.driver.cores", 4)
	.config("sql.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
	.config(
		"spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog"
	)
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/sonle/.sdkman/candidates/spark/3.4.0/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/sonle/.ivy2/cache
The jars for the packages stored in: /Users/sonle/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-01703b23-4e67-4cfb-81cf-bdd9cb29c396;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 215ms :: artifacts dl 11ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   

## Parquet is all the way
Log content here is stored on disk in **json format** which is not well-suited for reading purpose. 
Hence, we should load and write it down as **parquet format** since parquet is more toward to analytical workload. 

In [3]:
log_path = "/Users/sonle/Documents/GitHub/spark-playground/data/log_content"	
data_path = "/Users/sonle/Documents/GitHub/spark-playground/data/"
parquet_path = "parquet_log_content.parquet"

In [27]:
import os

os.path.exists(os.path.join(data_path, "log_content"))

True

In [5]:
# Just run this function one time
def read_log_content(
		data_paths: str,
		parquet_path: str
):
	"""
    Read and rename log content data in json format and 
    write it to disk in parquet format with partitioning option
    :param data_paths: data directory
    :param parquet_path: parquet file name
    :return: None
    """
	return (
		spark.read.json(f"{data_paths}/log_content/*.json")
		.select(
			col("_index").alias("Index"),
			col("_type").alias("Type"),
			col("_id").alias("Id"),
			col("_score").alias("Score"),
			col("_source.*"),
			to_date(
				regexp_extract(input_file_name(), r"\d{8}", 0),
				"yyyyMMdd"
			).alias("Date")
		)
		.write.parquet(
			path=f"{data_paths}/{parquet_path}",
			mode="overwrite",
			partitionBy="Date",
			compression="zstd"
		)
	)


read_log_content(data_path, parquet_path)

In [35]:
df = (
	spark.read.parquet(
		f"{data_path}/{parquet_path}"
	)
	.filter(col("Contract") != "0")
)


== Physical Plan ==
* Filter (3)
+- * ColumnarToRow (2)
   +- Scan parquet  (1)


(1) Scan parquet 
Output [9]: [Index#329, Type#330, Id#331, Score#332L, AppName#333, Contract#334, Mac#335, TotalDuration#336L, Date#337]
Batched: true
Location: InMemoryFileIndex [file:/Users/sonle/Documents/GitHub/spark-playground/data/parquet_log_content.parquet]
PartitionFilters: [isnotnull(Date#337), (cast(Date#337 as timestamp) >= 2022-04-01 00:00:00), (cast(Date#337 as timestamp) <= 2022-04-03 00:00:00)]
PushedFilters: [Not(EqualTo(Contract,0)), IsNotNull(Contract)]
ReadSchema: struct<Index:string,Type:string,Id:string,Score:bigint,AppName:string,Contract:string,Mac:string,TotalDuration:bigint>

(2) ColumnarToRow [codegen id : 1]
Input [9]: [Index#329, Type#330, Id#331, Score#332L, AppName#333, Contract#334, Mac#335, TotalDuration#336L, Date#337]

(3) Filter [codegen id : 1]
Input [9]: [Index#329, Type#330, Id#331, Score#332L, AppName#333, Contract#334, Mac#335, TotalDuration#336L, Date#337]
Condit

In [31]:
(
	spark.read.json(
		path=f"{data_path}/log_content"
	)
).show(5)

['2022-04-01', '2022-04-02', '2022-04-03']


In [32]:
spark.read.parquet(
	f"{data_path}/{parquet_path}/Date=2022-04-02"
).createOrReplaceTempView("log_content")

['/Users/sonle/Documents/GitHub/spark-playground/data//parquet_log_content.parquet/Date=2022-04-01',
 '/Users/sonle/Documents/GitHub/spark-playground/data//parquet_log_content.parquet/Date=2022-04-02',
 '/Users/sonle/Documents/GitHub/spark-playground/data//parquet_log_content.parquet/Date=2022-04-03']

In [34]:
df.createOrReplaceTempView("log_content")


== Physical Plan ==
* Filter (3)
+- * ColumnarToRow (2)
   +- Scan parquet  (1)


(1) Scan parquet 
Output [8]: [Index#312, Type#313, Id#314, Score#315L, AppName#316, Contract#317, Mac#318, TotalDuration#319L]
Batched: true
Location: InMemoryFileIndex [file:/Users/sonle/Documents/GitHub/spark-playground/data/parquet_log_content.parquet/Date=2022-04-01, ... 2 entries]
PushedFilters: [Not(EqualTo(Contract,0)), IsNotNull(Contract)]
ReadSchema: struct<Index:string,Type:string,Id:string,Score:bigint,AppName:string,Contract:string,Mac:string,TotalDuration:bigint>

(2) ColumnarToRow [codegen id : 1]
Input [8]: [Index#312, Type#313, Id#314, Score#315L, AppName#316, Contract#317, Mac#318, TotalDuration#319L]

(3) Filter [codegen id : 1]
Input [8]: [Index#312, Type#313, Id#314, Score#315L, AppName#316, Contract#317, Mac#318, TotalDuration#319L]
Condition : (NOT (Contract#317 = 0) AND isnotnull(Contract#317))


# ETL
The requirements at this part is that we need to extract the information of each contract in terms of total duration of five segments "TVDuration", "MovieDuration", "SportDuration",
"ChildDuration", "RelaxDuration".

The first option is to **aggregate sum function manually**, the second option is to use supported **pyspark pivot** method. 

Since we have 5 categories, we will need to rewrite some operations repeatedly which can take some time. For example:
```python
when(col("name") == "some_name", value).otherwise(value)
# or sum method in agg method
sum("columns").alias("columns_name")
```
Thankfully, pyspark functions support arguments unpacking `*exprs` so we can leverage this by creating lists of expressions before pasting them to pyspark functions.


In [57]:
 app_names = [
        "CHANNEL",
		"KPLUS",
        "VOD",
        "FIMS",
        "BHD",
        "SPORT",
        "CHILD",
        "RELAX",
    ]

column_names = [
        "TV",
        "TV",
        "Movie",
        "Movie",
        "Movie",
        "Sport",
        "Child",
        "Relax",
    ]
# whens = [
# 	when(col("AppName").isin(app_name), column_name).otherwise("")
# 	for app_name, column_name in zip(app_names, column_names)
# ]
# whens
import pyspark.sql.functions as F

whens_1 = F
for app_name, column_name in zip(app_names, column_names):
	whens_1 = whens_1.when(col("AppName") == app_name, column_name)
whens_1 = whens_1.otherwise("").alias("Type")
whens_1

	

Column<'CASE WHEN (AppName = CHANNEL) THEN TV WHEN (AppName = KPLUS) THEN TV WHEN (AppName = VOD) THEN Movie WHEN (AppName = FIMS) THEN Movie WHEN (AppName = BHD) THEN Movie WHEN (AppName = SPORT) THEN Sport WHEN (AppName = CHILD) THEN Child WHEN (AppName = RELAX) THEN Relax ELSE  END AS Type'>

In [94]:
def summarize_by_manual_pivot(
		df: DataFrame,
		app_names: list[str],
		column_names: list[str]
) -> DataFrame:
	"""
	Function to manually pivot data. 
	:param df: Dataframe, log content data
	:param app_names: application names in original data
	:param column_names: new column names after aggregation
	:return: new summarized data with required format
	"""
	
	whens = [when(col("AppName") == app_name, col("TotalDuration")).otherwise(lit(0)).alias(f"{column_name}")
			 for app_name, column_name in zip(app_names, column_names)
			 ]
	
	exprs = [sum(x).alias(f"{x}") for x in column_names]

	return (
		df
		.select(
			col("Contract"),
			*whens)
		.groupby("Contract")
		.agg(*exprs)
		.orderBy("TVDuration", ascending=False)
	)


In [95]:
%%time
summarize_by_manual_pivot(df, app_names=app_map, column_names=columns_name)

+---------+-----------+-------------+-------------+-------------+-------------+
| Contract|TVDurartion|MovieDuration|SportDuration|ChildDuration|RelaxDuration|
+---------+-----------+-------------+-------------+-------------+-------------+
|HPFD01358|    87334.0|         null|         null|      13491.0|         null|
|BND016514|  1741303.0|         null|     218863.0|         null|         null|
|NTFD09088|   854026.0|         null|         null|         null|         null|
|TNFD18206|   790149.0|         null|         null|      60996.0|         15.0|
|HNH585980|   650303.0|         null|         null|         null|         null|
|HNH898380|   753701.0|         null|         null|      30749.0|         null|
|DAFD53097|  1494819.0|         null|         null|         null|         null|
|HNH619464|    47127.0|       5297.0|         null|       2132.0|        198.0|
|NBFD09044|   136249.0|      63765.0|         null|         null|         null|
|HNH729854|   689090.0|      72982.0|   

In [34]:
spark.sql(
	sqlQuery="""
	select
		distinct  AppName
		
	from log_content 
	"""
).show()

+-------+
|AppName|
+-------+
|  KPLUS|
|  RELAX|
|  CHILD|
|   FIMS|
|CHANNEL|
|  SPORT|
|    VOD|
|    APP|
|    BHD|
+-------+


In [62]:

(
	df
	.select(
		col("Contract"),
		col("TotalDuration"),
		whens_1
	)
	.filter(
        (col("Contract") != "0") & (col("Contract") != "") & (col("Type") != "Unknown")
	)
	.groupBy("Contract")
	.pivot("Type")
	.sum("TotalDuration")
	.show()
)



+---------+-----+------+-----+-----+-------+
| Contract|Child| Movie|Relax|Sport|     TV|
+---------+-----+------+-----+-----+-------+
|HNH433136| null|  4886| null| null|2034901|
|NTFD81659| null|  2195| null| null|2280405|
|VTFD30952| 3988|  null| null| null| 789944|
|SGH317545| null|  null| null| null| 294590|
|SGH382957| 2554|136065| null| null| 862148|
|BTFD16422| null|  null| null| null|1491852|
|DNFD55835|   76|    23| null| null|  62855|
|HND443645| 5479| 68502| 2730| null| 528563|
|QNAAA3126| null|  null| null| null| 924909|
|SGH975310| null| 19136| null| null|1269159|
|HNH655987| null|  null| null| null| 232239|
|HNH853741| null|  null| null| null| 322787|
|DLFD15794|35718|  null| null| null| 122067|
|VPFD10557| null|  null| null| null|  78463|
|SGH294690| null|  null|   20| null| 788329|
|SGH630137| null|  9739| null| null| 412279|
|TNFD02599| 4206|  6330| null| null| 281203|
|QNFD20415| null| 15357| null| null|  92003|
|GLAAA0389| null|  null| null| null| 140277|
|HNH882752

In [60]:
whens_1

Column<'CASE WHEN (AppName = CHANNEL) THEN TV WHEN (AppName = KPLUS) THEN TV WHEN (AppName = VOD) THEN Movie WHEN (AppName = FIMS) THEN Movie WHEN (AppName = BHD) THEN Movie WHEN (AppName = SPORT) THEN Sport WHEN (AppName = CHILD) THEN Child WHEN (AppName = RELAX) THEN Relax ELSE  END AS Type'>

In [27]:
df.show() # HNH433136,null,4886,null,null,2034901


+-------+-------+--------------------+-----+-------+---------+------------+-------------+----------+
|  Index|   Type|                  Id|Score|AppName| Contract|         Mac|TotalDuration|      Date|
+-------+-------+--------------------+-----+-------+---------+------------+-------------+----------+
|history|channel|AYB79qCpa1FFivsGZAN3|    0|CHANNEL|HNH757497|405BD8A648C6|        77713|2022-04-30|
|history|channel|AYB79qCpa1FFivsGZAN8|    0|CHANNEL|LDFD14107|802BF9BE4376|        77712|2022-04-30|
|history|channel|AYB79qCga1FFivsGZAHn|    0|CHANNEL|SGH538814|802BF95B0E77|        78677|2022-04-30|
|history|channel|AYB79qCpa1FFivsGZAOI|    0|CHANNEL|HND368630|4CEBBD201707|        77706|2022-04-30|
|history|channel|AYB79qCpa1FFivsGZAON|    0|CHANNEL|BDFD59880|0C96E68E61CD|        77704|2022-04-30|
|history|channel|AYB79qCga1FFivsGZAHx|    0|CHANNEL|HPFD87960|B84DEE853CE4|        78671|2022-04-30|
|history|channel|AYB79qCpa1FFivsGZAOb|    0|CHANNEL|PYFD06964|0C96E6E85735|        77695|20

In [26]:
(
	df
		.groupby("Contract")
		.pivot("Type")
		.sum("TotalDuration")
		.show(truncate=False)
)

+---------+----+----+-------+-----+----+-----+-----+-----+------+
|Contract |app |bhd |channel|child|fims|kplus|relax|sport|vod   |
+---------+----+----+-------+-----+----+-----+-----+-----+------+
|DAD054623|null|null|282136 |null |null|null |null |null |null  |
|NAFD04728|null|null|195244 |null |null|null |null |44   |null  |
|VTH008162|null|null|342727 |null |null|null |null |null |null  |
|HPH005457|null|null|1542340|null |null|null |null |null |null  |
|HNH526745|null|null|43686  |null |null|null |null |null |null  |
|DAFD69633|null|null|69327  |null |null|null |null |null |41281 |
|QND027862|null|null|111895 |null |null|null |null |null |null  |
|AGFD35631|null|null|862753 |null |null|null |329  |null |null  |
|BNFD57960|null|null|116075 |null |null|null |null |null |null  |
|BNFD70630|null|null|961771 |null |null|null |null |null |null  |
|SGH032371|null|null|91530  |null |null|null |null |null |32596 |
|SGJ002494|null|null|120021 |null |null|null |null |null |253   |
|SGD611354

In [102]:
def summarize_by_supported_pivot(
		df: DataFrame,
		app_names: list[str],
		column_names: list[str]
) -> DataFrame:
	"""
	Function to pivot data using supported pyspark one
	:param df: 
	:param app_names: 
	:param column_names: 
	:return: 
	"""
	exprs = [col(x).alias(f"{y}") for x, y in zip(app_names, column_names)]

	return (
		df
		.groupby("Contract")
		.pivot("AppName", app_names)
		.sum("TotalDuration")
		.select(
			col("Contract"),
			*exprs
		)
		.orderBy("TVDuration", ascending=False)
	)

In [103]:
%%time
summarize_by_supported_pivot(df, app_names=app_names, column_names=column_names)


+---------+----------+-------------+-------------+-------------+-------------+
| Contract|TVDuration|MovieDuration|SportDuration|ChildDuration|RelaxDuration|
+---------+----------+-------------+-------------+-------------+-------------+
|NBFD21210|  26445048|       406535|       647810|         1427|           32|
|BDD044379|  26085282|        26639|         null|         1067|          386|
|BDFD58200|  24655135|         null|         null|         null|           93|
|BLFD00046|  15783646|       889257|       707309|        77916|        46116|
|DNFD92311|  15068109|       951556|       573862|       636809|       334250|
|    PAYTV|  13490019|       239052|       455106|       267162|        11452|
|HNH718815|  13015472|       139572|         null|       119213|        20286|
|SGH336949|  12825697|         null|         null|          109|        87815|
|SGFD52266|  11046008|       544069|         null|       113266|        14801|
|SGH952911|  10373154|       402057|         null|  